In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # importe ESIOS_TOKEN, AEMET_TOKEN, etc.
os.chdir("..")

In [ ]:
!pip install tensorflow

In [7]:
# src/models/dl_training.py

import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def train_and_save(model, X_train, y_train, X_val, y_val,
                   models_dir, zone, horizon, name,
                   epochs=50, batch_size=32):
    """
    Entraîne un modèle Keras sur X_train/y_train, valide sur X_val/y_val,
    sauvegarde les poids et retourne l'historique.
    """
    os.makedirs(models_dir, exist_ok=True)
    filepath = os.path.join(models_dir, f"{zone}_{horizon}_{name}.h5")
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True)
    ]
    hist = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=2
    )
    return hist, filepath


In [8]:
# src/models/evaluation.py

import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_regression(model, X_test, y_test, pred_series=None):
    """
    Calcule MAE, RMSE, MAPE pour un modèle sklearn ou SARIMAX.
    - model       : un objet sklearn ou statsmodels (SARIMAXResults)
    - X_test      : DataFrame X_test (None si pred_series fourni)
    - y_test      : Series y_test
    - pred_series : Series de prédictions (optionnel)
    """
    if pred_series is None:
        y_pred = model.predict(X_test)
    else:
        y_pred = pred_series

    mae  = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")
    return {"MAE": mae, "RMSE": rmse, "MAPE": mape}


In [ ]:
# 06_modeling_dl.ipynb — Deep Learning Models

# Cellule 1. Imports
import pandas as pd
from pathlib import Path
from src.models.dl_utils import create_sequences, scale_data
from src.models.dl_models import build_lstm, build_gru, build_cnn, build_cnn_lstm
from src.models.dl_training import train_and_save
from src.models.evaluation import evaluate_regression
import numpy as np


In [10]:
# !pip install yaml

In [11]:
# Cellule 2. Configuration & chemins
# def load_config(config_path: str = "config.yaml") -> dict:
#     """
#     Lit le fichier YAML de configuration et renvoie un dict.
#     """
#     with open(config_path, "r") as f:
#         cfg = yaml.safe_load(f)

#     return cfg

# cfg = load_config("config.yaml")
# processed_dir = Path(cfg["paths"]["data"]["processed"])
# models_dir    = Path("models/dl")
# metrics = []
# zones = list(cfg["geo_zones"].values())


processed_dir = Path("./data/processed")
models_dir    = Path("./models/dl")
metrics = []

# liste des zones
zones = [p.stem.replace("_processed_hourly","")
         for p in processed_dir.glob("*_processed_hourly.parquet")]

In [12]:
zones

['Baleares',
 'Canarias',
 'Ceuta',
 'El_Hierro',
 'Gran_canaria',
 'Lanzarote_Fuerteventura',
 'La_Gomera',
 'La_Palma',
 'Melilla',
 'nacional',
 'Peninsule_Iberique',
 'Tenerife']

In [13]:
# Cellule 3. Hyperparamètres
LOOKBACK = 24  # ex. 24 h
HORIZON  = 1   # 1 pas à prévoir
BATCH    = 32
EPOCHS   = 50


In [14]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Cellule 4. Boucle par zone et horizon
for zone in zones:
    for horizon in ["daily", "hourly"]:
        # Charger les données
        file = processed_dir/f"{zone}_processed_{horizon}.parquet"
        df = pd.read_parquet(file)

        # Sélection features
        feature_cols = [c for c in df.columns if c != "demand"]

        # Créer séquences
        X, y = create_sequences(df, "demand", feature_cols, LOOKBACK, HORIZON)
        # Split 80/20
        split = int(len(X)*0.8)
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]
        # Scale features
        X_train, X_val, scaler = scale_data(X_train, X_val)

        input_shape = (LOOKBACK, len(feature_cols))

        # Modèles à tester
        models = {
            "LSTM":      build_lstm(input_shape, HORIZON),
            "GRU":       build_gru(input_shape, HORIZON),
            "CNN":       build_cnn(input_shape, HORIZON),
            "CNN-LSTM":  build_cnn_lstm(input_shape, HORIZON)
        }

        for name, model in models.items():
            print(f"Training {name} for {zone} {horizon}...")

            hist, filepath = train_and_save(
                model, X_train, y_train, X_val, y_val,
                models_dir, zone, horizon, name,
                epochs=EPOCHS, batch_size=BATCH
            )
            # Prédictions pour évaluation
            y_pred = model.predict(X_val)
            # Flatten si besoin
            y_val_flat = y_val.ravel()
            y_pred_flat = y_pred.ravel()
            score = evaluate_regression(None, None, pd.Series(y_val_flat), pd.Series(y_pred_flat))
            metrics.append({
                "zone": zone,
                "horizon": horizon,
                "model": name,
                **score
            })


In [28]:
# Cellule 4. Boucle par zone et horizon v2
for zone in zones:
    for horizon in ["daily", "hourly"]:
        # Charger les données
        file = processed_dir/f"{zone}_processed_{horizon}.parquet"
        df = pd.read_parquet(file)

        # Sélection features
        feature_cols = [c for c in df.columns if c != "demand"]

        # Créer séquences
        X, y = create_sequences(df, "demand", feature_cols, LOOKBACK, HORIZON)
        # Split 80/20
        split = int(len(X)*0.8)
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]
        # Scale features
        X_train, X_val, scaler = scale_data(X_train, X_val)

        input_shape = (LOOKBACK, len(feature_cols))

        # Modèles à tester
        models = {
            "LSTM":      build_lstm(input_shape, HORIZON),
            "GRU":       build_gru(input_shape, HORIZON),
            "CNN":       build_cnn(input_shape, HORIZON),
            "CNN-LSTM":  build_cnn_lstm(input_shape, HORIZON)
        }

        for name, model in models.items():
            print(f"Training {name} for {zone} {horizon}...")

            model_file = models_dir / f"{zone}_{horizon}_{name}.h5"

            print(f"Model file: {model_file}")

            if os.path.exists(model_file):
              print(f"\n \n Model for {zone} {horizon} already exists. Loading...")
              # Load the model
              from tensorflow.keras.models import load_model

              model = load_model(model_file, custom_objects={"mse": tensorflow.keras.losses.MeanSquaredError()})
            else:
              hist, filepath = train_and_save(
                  model, X_train, y_train, X_val, y_val,
                  models_dir, zone, horizon, name,
                  epochs=EPOCHS, batch_size=BATCH
              )

            # Prédictions pour évaluation
            y_pred = model.predict(X_val)
            # Flatten si besoin
            y_val_flat = y_val.ravel()
            y_pred_flat = y_pred.ravel()
            score = evaluate_regression(None, None, pd.Series(y_val_flat), pd.Series(y_pred_flat))
            metrics.append({
                "zone": zone,
                "horizon": horizon,
                "model": name,
                **score
            })


Training LSTM for Baleares daily...
Model file: models/dl/Baleares_daily_LSTM.h5

 
 Model for Baleares daily already exists. Loading...
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
MAE: 16287.23, RMSE: 17064.92, MAPE: 99.53%
Training GRU for Baleares daily...
Model file: models/dl/Baleares_daily_GRU.h5

 
 Model for Baleares daily already exists. Loading...


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
MAE: 16089.98, RMSE: 16876.76, MAPE: 97.42%
Training CNN for Baleares daily...
Model file: models/dl/Baleares_daily_CNN.h5

 
 Model for Baleares daily already exists. Loading...


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
MAE: 1923.88, RMSE: 3082.69, MAPE: 61.83%
Training CNN-LSTM for Baleares daily...
Model file: models/dl/Baleares_daily_CNN-LSTM.h5

 
 Model for Baleares daily already exists. Loading...


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
MAE: 16094.13, RMSE: 16880.72, MAPE: 97.47%
Training LSTM for Baleares hourly...
Model file: models/dl/Baleares_hourly_LSTM.h5

 
 Model for Baleares hourly already exists. Loading...


329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
MAE: 213.16, RMSE: 279.42, MAPE: inf%
Training GRU for Baleares hourly...
Model file: models/dl/Baleares_hourly_GRU.h5

 
 Model for Baleares hourly already exists. Loading...


...

1316/1316 - 12s - 9ms/step - loss: 959.7712 - val_loss: 370.0532
Epoch 19/50
1316/1316 - 9s - 7ms/step - loss: 959.2567 - val_loss: 384.3177
Epoch 20/50


1316/1316 - 10s - 8ms/step - loss: 936.1118 - val_loss: 355.7555
Epoch 21/50
1316/1316 - 9s - 7ms/step - loss: 938.8848 - val_loss: 360.4884
Epoch 22/50
1316/1316 - 9s - 7ms/step - loss: 933.6224 - val_loss: 358.4496
Epoch 23/50
1316/1316 - 9s - 7ms/step - loss: 922.5334 - val_loss: 369.6439
Epoch 24/50
1316/1316 - 9s - 7ms/step - loss: 925.3736 - val_loss: 392.4392
Epoch 25/50


1316/1316 - 10s - 8ms/step - loss: 907.5148 - val_loss: 344.6303
Epoch 26/50
1316/1316 - 8s - 6ms/step - loss: 907.8999 - val_loss: 389.2809
Epoch 27/50
1316/1316 - 11s - 8ms/step - loss: 894.8198 - val_loss: 388.6652
Epoch 28/50
1316/1316 - 9s - 7ms/step - loss: 896.6066 - val_loss: 346.5423
Epoch 29/50
1316/1316 - 10s - 7ms/step - loss: 897.0717 - val_loss: 352.5874
Epoch 30/50


1316/1316 - 9s - 7ms/step - loss: 903.4632 - val_loss: 339.1966
Epoch 31/50


1316/1316 - 10s - 8ms/step - loss: 880.2463 - val_loss: 328.0457
Epoch 32/50
1316/1316 - 11s - 8ms/step - loss: 883.7358 - val_loss: 335.3832
Epoch 33/50
1316/1316 - 10s - 8ms/step - loss: 873.9390 - val_loss: 366.7182
Epoch 34/50
1316/1316 - 9s - 7ms/step - loss: 885.9275 - val_loss: 347.6543
Epoch 35/50
1316/1316 - 12s - 9ms/step - loss: 868.5195 - val_loss: 364.2910
Epoch 36/50
1316/1316 - 9s - 7ms/step - loss: 869.0165 - val_loss: 344.6199
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
MAE: 7.58, RMSE: 18.11, MAPE: inf%


In [29]:
# Cellule 5. Sauvegarde des métriques
pd.DataFrame(metrics).to_csv("outputs/reports/all_zones_dl_metrics.csv", index=False)


In [30]:
metrics


[{'zone': 'Baleares',
  'horizon': 'daily',
  'model': 'LSTM',
  'MAE': 16091.97755456543,
  'RMSE': np.float64(16878.671891861348),
  'MAPE': np.float64(97.44540039850904)},
 {'zone': 'Baleares',
  'horizon': 'daily',
  'model': 'GRU',
  'MAE': 16089.97621475593,
  'RMSE': np.float64(16876.763862551037),
  'MAPE': np.float64(97.42404897308103)},
 {'zone': 'Baleares',
  'horizon': 'daily',
  'model': 'CNN',
  'MAE': 1923.8830882138036,
  'RMSE': np.float64(3082.694653600928),
  'MAPE': np.float64(61.83346342410639)},
 {'zone': 'Baleares',
  'horizon': 'daily',
  'model': 'CNN-LSTM',
  'MAE': 16094.127564955677,
  'RMSE': np.float64(16880.721716105534),
  'MAPE': np.float64(97.46833621982212)},
 {'zone': 'Baleares',
  'horizon': 'daily',
  'model': 'LSTM',
  'MAE': 16287.228626495362,
  'RMSE': np.float64(17064.92375726007),
  'MAPE': np.float64(99.52841435951278)},
 {'zone': 'Baleares',
  'horizon': 'daily',
  'model': 'GRU',
  'MAE': 16089.97621475593,
  'RMSE': np.float64(16876.76386